In [1]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %                   SCF-TB - PROXY APPLICATION                      %
# %                   A.M.N. Niklasson, M. Kulichenko. T1, LANL       %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# % Total Energy Function:                                            %
# % E = 2Tr[H0(D-D0)] + (1/2)*sum_i U_i q_i^2 +                       %
# %      + (1/2)sum_{i,j (i!=j)} q_i C_{ij} q_j - Efield*dipole       %
# % dipole = sum_i R_{i} q_i                                          %
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

import torch
import warnings
import logging
import os
os.environ["TORCHDYNAMO_DISABLE"] = "1"  # hard-disable capture

import numpy as np
import matplotlib.pyplot as plt

from dftorch.SCF import SCFx, SCFx_batch, scf_x_os
from dftorch.Energy import Energy

from dftorch.Constants import Constants
from dftorch.Structure import Structure, StructureBatch
from dftorch.MD import MDXL, MDXLBatch
from dftorch.ESDriver import ESDriver, ESDriverBatch
from dftorch.Fermi_PRT import Fermi_PRT_batch
from dftorch.Spin import get_spin_energy, get_h_spin

from dftorch.Forces import Forces



from typing import Optional, Dict, Any, Tuple

### Configure torch and torch.compile ###
# Silence warnings and module logs
warnings.filterwarnings("ignore")
os.environ["TORCH_LOGS"] = ""               # disable PT2 logging
os.environ["TORCHINDUCTOR_VERBOSE"] = "0"
os.environ["TORCHDYNAMO_VERBOSE"] = "0"
logging.getLogger("torch.fx").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.symbolic_shapes").setLevel(logging.CRITICAL)
logging.getLogger("torch.fx.experimental.recording").setLevel(logging.CRITICAL)
# Enable dynamic shape capture for dynamo
torch._dynamo.config.capture_dynamic_output_shape_ops = True
# default data type
torch.set_default_dtype(torch.float64)
# to disable torchdynamo completely. Faster for smaller systems and single-point calculations.

torch.cuda.empty_cache()

In [11]:
#%%time
dftorch_params = {
    'coul_method': '!PME', # 'FULL' for full coulomb matrix, 'PME' for PME method
    'Coulomb_acc': 1e-5,   # Coulomb accuracy for full coulomb calcs or t_err for PME
    'cutoff': 12.0,        # Coulomb cutoff
    'PME_order': 4,        # Ignored for FULL coulomb method

    'SCF_MAX_ITER': 100,    # Maximum number of SCF iterations
    'SCF_TOL': 1e-6,       # SCF convergence tolerance on density matrix
    'SCF_ALPHA': 0.1,      # Scaled delta function coefficient. Acts as linear mixing coefficient used before Krylov acceleration starts.

    'KRYLOV_MAXRANK': 20,  # Maximum Krylov subspace rank
    'KRYLOV_TOL': 1e-6,    # Krylov subspace convergence tolerance in SCF
    'KRYLOV_TOL_MD': 1e-4, # Krylov subspace convergence tolerance in MD SCF
    'KRYLOV_START': 10,     # Number of initial SCF iterations before starting Krylov acceleration
                }
                
# Initial data, load atoms and coordinates, etc in COORD.dat
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'xpu'
# filename = 'COORD.xyz'
# LBox = torch.tensor([10,15,15], device=device)

# filename = ['C1.xyz']
# LBox = torch.tensor([[30,30,30]], device=device)

filename = '8RNT.xyz'
LBox = None
#LBox = torch.tensor([30,30,30], device=device)

# filename = 'H2.xyz'
# LBox = torch.tensor([30,30,30], device=device)

# Create constants container. Set path to SKF files.
const = Constants(filename,
                  #'/home/maxim/Projects/DFTB/DFTorch/tests/sk_orig/ptbp/complete_set',
                  'C:\\000_MyFiles\\Programs\\DFTorch\\tests\\sk_orig\\mio-1-1\\mio-1-1\\',
                  magnetic_hubbard_ldep=False
                  ).to(device)

structure1 = Structure(filename, LBox, const, charge=0, Te=500.0, device=device)

es_driver = ESDriver(dftorch_params, electronic_rcut=8.0, repulsive_rcut=6.0, device=device)
es_driver(structure1, const, do_scf=False)
#es_driver.calc_forces(structure1, const)


int64
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-C.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-N.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-O.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-S.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\H-Zn.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-C.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-N.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-O.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-S.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\C-Zn.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\N-H.skf
C:\000_MyFiles\Programs\DFTorch\tests\sk_orig\mio-1-1\mio-1-1\N-C.skf
C:\000_MyFil

In [15]:
structure1.RX.max(), structure1.RX.min(),\
structure1.RY.max(), structure1.RY.min(),\
structure1.RZ.max(), structure1.RZ.min()

(tensor(35.5570),
 tensor(-3.5470),
 tensor(32.6650),
 tensor(-8.3650),
 tensor(23.4300),
 tensor(-9.9220))

In [4]:
structure1.Q

tensor([[ 0.2837, -0.4187,  0.4365,  ..., -0.1915,  0.2159, -0.1085],
        [ 0.0665, -0.0528, -0.0509,  ..., -0.0186,  0.0473, -0.0600],
        [-0.0005,  0.0010, -0.0028,  ..., -0.0178,  0.0657, -0.0580],
        ...,
        [ 0.0665,  0.1383,  0.1987,  ...,  0.1070,  0.0899,  0.0332],
        [ 0.0729,  0.1522,  0.2108,  ...,  0.2049,  0.1636,  0.0729],
        [ 0.1598,  0.2498,  0.1712,  ...,  0.3482,  0.4106,  0.2905]])

In [ ]:
# structure1.H, structure.Hcoul, structure1.Hdipole, structure1.KK, structure1.D, \
# structure1.q, structure1.f, \
# structure1.mu0, structure1.e_coul_tmp, structure1.f_coul, structure1.dq_p1 = \
H, Hcoul, Hdipole, KK, D, Q, q_spin_atom, q_tot_atom, q_spin_sr, net_spin_sr, f, mu0, Ecoul, forces1, dq_p1 = \
    scf_x_os(structure1.el_per_shell, structure1.shell_types, structure1.n_shells_per_atom, const.shell_dim, const.w,
    dftorch_params, structure1.RX, structure1.RY, structure1.RZ, structure1.lattice_vecs,
    structure1.Nats, structure1.Nocc, structure1.n_orbitals_per_atom, structure1.Znuc, structure1.TYPE,
    structure1.Te,
    structure1.Hubbard_U,
    structure1.D0,
    structure1.H0, structure1.S, structure1.Z, structure1.e_field, structure1.C, structure1.req_grad_xyz)

### Do SCF ###
  Initial DM_Fermi
tensor(0.1344) tensor([0.1344, 0.1344])
tensor([-0.6523,  1.0760, -0.0810, -0.1752, -0.1155, -0.0878, -0.6124,  1.0622,
        -0.0852, -0.1282, -0.1306, -0.2378, -0.6129,  1.0672, -0.0898, -0.1804,
        -0.0946, -0.2239, -0.6459,  1.0885, -0.0937, -0.0574, -0.0747, -0.2344]) tensor(-0.3200)

Starting cycle
Iter 1
Res = 0.146557876, dEc = 0.408306172, t = 0.0 s

Iter 2
Res = 0.128485310, dEc = 0.040030178, t = 0.0 s

Iter 3
Res = 0.113092713, dEc = 0.031261869, t = 0.0 s

Iter 4
Res = 0.099932612, dEc = 0.024463711, t = 0.0 s

Iter 5
Res = 0.088633028, dEc = 0.019177987, t = 0.0 s

Iter 6
Res = 0.078885741, dEc = 0.015057194, t = 0.0 s

Iter 7
Res = 0.070436151, dEc = 0.011836579, t = 0.0 s

Iter 8
Res = 0.063074521, dEc = 0.009313737, t = 0.0 s

Iter 9
Res = 0.056628424, dEc = 0.007333391, t = 0.0 s

Iter 10
Res = 0.050956277, dEc = 0.005776018, t = 0.0 s

Iter 11
  rank: 0, Fel = 0.001147
  rank: 1, Fel = 0.000105
  rank: 2, Fel = 0.000025
  rank

In [38]:
q_tot_atom

tensor([ 0.16361, -0.08991, -0.07370], device='cuda:0')

In [39]:
H_spin = get_h_spin(structure1.TYPE, net_spin_sr, const.w, structure1.n_shells_per_atom, structure1.shell_types)
H_spin = 0.5 * structure1.S * H_spin.unsqueeze(0).expand(2, -1, -1) * torch.tensor([[[1]],[[-1]]], device=H_spin.device)

In [40]:
torch.set_printoptions(precision=5)
ev_A_to_ha_B = 1.0 / 51.422067#0.0367493/1.88973

In [41]:
Ftot, Fcoul, Fband0, Fdipole, FPulay, FScoul, FSdipole, Frep = Forces(H-H_spin, structure1.Z, structure1.C,
                        D, structure1.D0,
                        structure1.dH0, structure1.dS, structure1.dCC, structure1.dVr,
                        structure1.e_field, structure1.Hubbard_U, q_tot_atom,
                        structure1.RX, structure1.RY, structure1.RZ,
                        structure1.Nats, const, structure1.TYPE)

In [42]:
Ftot, Fcoul, Fband0, Fdipole, FPulay, FScoul, FSdipole, Frep = Forces(H, structure1.Z, structure1.C,
                        D, structure1.D0,
                        structure1.dH0, structure1.dS, structure1.dCC, structure1.dVr,
                        structure1.e_field, structure1.Hubbard_U, q_tot_atom,
                        structure1.RX, structure1.RY, structure1.RZ,
                        structure1.Nats, const, structure1.TYPE)

In [43]:
e_elec_tot, e_band0, e_coul, e_dipole, e_entropy, s_ent = Energy(
                structure1.H0, structure1.Hubbard_U, structure1.e_field,
                structure1.D0, structure1.C, structure1.dq_p1, D, q_tot_atom,
                structure1.RX, structure1.RY, structure1.RZ, f, structure1.Te)

e_spin = get_spin_energy(structure1.TYPE, net_spin_sr, const.w, structure1.n_shells_per_atom)

e_elec_tot + e_spin + structure1.e_repulsion - e_entropy,\
e_band0/27.21138625, e_coul/27.21138625, e_spin/27.21138625, e_entropy, structure1.e_repulsion/27.21138625

(tensor(-62.46010, device='cuda:0'),
 tensor(-2.29312, device='cuda:0'),
 tensor(0.00086, device='cuda:0'),
 tensor(0., device='cuda:0'),
 tensor(-1.93909e-07, device='cuda:0'),
 tensor(-0.00311, device='cuda:0'))

In [49]:
const.w[structure1.TYPE]

tensor([-0.61770, -1.95106, -1.95106], device='cuda:0')

In [50]:
net_spin_sr

tensor([0., 0., 0., 0.], device='cuda:0')

In [16]:
net_spin = q_spin_atom[0] - q_spin_atom[1]

atom_ids = torch.repeat_interleave(torch.arange(len(structure1.n_orbitals_per_atom), device=structure1.H0.device), structure1.n_orbitals_per_atom) # Generate atom index for each orbital

FSspinA = torch.zeros((3, structure1.Nats), dtype=D.dtype, device=D.device)
factor = (net_spin * const.w[structure1.TYPE])*2
tmp = ((torch.tensor([[[1]],[[-1]]], device=H_spin.device) * D).unsqueeze(1)*structure1.dS.unsqueeze(0)).sum(0)
dS_times_D = tmp*factor[atom_ids].unsqueeze(-1)
dDS_XYZ_row_sum = torch.sum(dS_times_D, dim = 2) # sum of elements in each row
FSspinA.scatter_add_(1, atom_ids.expand(3, -1), dDS_XYZ_row_sum) # sums elements from DS into q based on number of AOs, e.g. x4 p orbs for carbon or x1 for hydrogen
dDS_XYZ_col_sum = torch.sum(dS_times_D, dim=1)
FSspinA.scatter_add_(1, atom_ids.expand(3, -1), -dDS_XYZ_col_sum)


RuntimeError: The size of tensor a (20) must match the size of tensor b (1772) at non-singleton dimension 0

In [52]:
q_spin_atom

tensor([[ 0.08181, -0.04496, -0.03685],
        [ 0.08181, -0.04496, -0.03685]], device='cuda:0')

In [53]:
net_spin

tensor([0., 0., 0.], device='cuda:0')

In [54]:
(Fband0 + FPulay + Fcoul + FScoul + (FSspinA)/2 + Frep)*ev_A_to_ha_B

tensor([[ 0.01705,  0.00148, -0.01853],
        [ 0.01946,  0.00045, -0.01990],
        [ 0.02223, -0.01885, -0.00338]], device='cuda:0')

In [51]:
SIHD = 4 * structure1.Z @ structure1.Z.T @ H_spin @ D # removes S factor from H
FSpin = torch.zeros((3, structure1.Nats), dtype=D.dtype, device=D.device)
# Collapse alpha and beta channels if present: dS @ (SIHD[0] + SIHD[1]) == dS @ SIHD.sum(0)
if SIHD.dim() == 2: # cs
    TMP = -(structure1.dS @ SIHD).diagonal(offset=0, dim1=1, dim2=2)
else: # os
    TMP = -0.5*torch.matmul(structure1.dS.unsqueeze(0), SIHD.unsqueeze(1)).sum(0).diagonal(offset=0, dim1=1, dim2=2) # (2, 3, 4, 4)

FSpin.scatter_add_(1, atom_ids.expand(3, -1), TMP) # sums elements from DS into q based on number of AOs, e.g. x4 p orbs for carbon or x1 for hydrogen


tensor([[-4.12974e-03,  5.67753e-02, -9.43059e-02,  6.20695e+03, -6.20802e+03],
        [-3.84007e-02, -6.44013e-02,  4.43439e-02, -9.59547e-12,  1.02234e-11],
        [ 1.03256e-01,  5.06330e-02,  7.13451e-02, -3.69214e-12,  3.46930e-12]],
       device='cuda:0')

In [ ]:
net_spin = q_spin_atom[0] - q_spin_atom[1]

atom_ids = torch.repeat_interleave(torch.arange(len(structure1.n_orbitals_per_atom), device=structure1.H0.device), structure1.n_orbitals_per_atom) # Generate atom index for each orbital

FSspinA = torch.zeros((3, structure1.Nats), dtype=D.dtype, device=D.device)
factor = (net_spin * const.w[structure1.TYPE][:,0,0])*2
dS_times_D = D[0]*structure1.dS*factor[atom_ids].unsqueeze(-1)
dDS_XYZ_row_sum = torch.sum(dS_times_D, dim = 2) # sum of elements in each row
FSspinA.scatter_add_(1, atom_ids.expand(3, -1), dDS_XYZ_row_sum) # sums elements from DS into q based on number of AOs, e.g. x4 p orbs for carbon or x1 for hydrogen
dDS_XYZ_col_sum = torch.sum(dS_times_D, dim=1)
FSspinA.scatter_add_(1, atom_ids.expand(3, -1), -dDS_XYZ_col_sum)


FSspinB = torch.zeros((3, structure1.Nats), dtype=D.dtype, device=D.device)
dS_times_D = D[1]*structure1.dS*factor[atom_ids].unsqueeze(-1)
dDS_XYZ_row_sum = torch.sum(dS_times_D, dim = 2) # sum of elements in each row
FSspinB.scatter_add_(1, atom_ids.expand(3, -1), dDS_XYZ_row_sum) # sums elements from DS into q based on number of AOs, e.g. x4 p orbs for carbon or x1 for hydrogen
dDS_XYZ_col_sum = torch.sum(dS_times_D, dim=1)
FSspinB.scatter_add_(1, atom_ids.expand(3, -1), -dDS_XYZ_col_sum)

(FSspinA - FSspinB)/2


tensor([[-2.40977e-02, -2.91311e-02,  5.32288e-02, -1.03848e+00,  1.03848e+00],
        [-8.47930e-03,  3.32467e-02, -2.47675e-02, -1.96382e-14,  1.96382e-14],
        [ 6.74009e-02, -2.64830e-02, -4.09179e-02, -9.07479e-15,  9.07479e-15]],
       device='cuda:0')

In [53]:
(Fband0 + FPulay_H0 + FSpin + (FSspinA - FSspinB)/2)*ev_A_to_ha_B

NameError: name 'FPulay_H0' is not defined

In [140]:
(Fband0 + FPulay + Fcoul + FScoul + FSpin + (FSspinA - FSspinB)/2 + Frep)*ev_A_to_ha_B

tensor([[ 0.0298778, -0.0126018, -0.0172760],
        [ 0.0298611, -0.0130558, -0.0168053],
        [-0.0318020,  0.0057070,  0.0260950]], device='cuda:0')

In [133]:
F_elec = (Fband0 + FPulay + FSpin + (FSspinA - FSspinB)/2)*ev_A_to_ha_B
F_elec

tensor([[ 0.0242062,  0.0288595, -0.0530656],
        [ 0.0183030,  0.0354797, -0.0537827],
        [-0.1241958,  0.0801932,  0.0440027]], device='cuda:0')

In [102]:
((FSspinA - FSspinB)/2)*ev_A_to_ha_B

tensor([[-5.4654351e-04,  4.9840084e-04,  4.8142672e-05],
        [-4.1699935e-04,  4.4899663e-04, -3.1997284e-05],
        [ 2.7416626e-03, -1.3515164e-03, -1.3901462e-03]], device='cuda:0')

In [130]:
ev_A_to_ha_B, 1.0 / 51.422067

(0.019446852195816332, 0.019446903991626786)

In [151]:
(Fcoul + FScoul + FPulay-FPulay_H0)*ev_A_to_ha_B

tensor([[-0.0017283,  0.0035588, -0.0057248],
        [-0.0011630,  0.0042988, -0.0058320],
        [ 0.0112714,  0.0086126,  0.0042479]], device='cuda:0')

In [99]:
(FSspinA - FSspinB)/2*ev_A_to_ha_B

tensor([[-5.4654351e-04,  4.9840084e-04,  4.8142672e-05],
        [-4.1699935e-04,  4.4899663e-04, -3.1997284e-05],
        [ 2.7416626e-03, -1.3515164e-03, -1.3901462e-03]], device='cuda:0')

In [ ]:
'''
CoulPot = C*q;  % Factor of 2 or 1/2 or +/-
FScoul = zeros(3,Nats); % Coulomb force FSCOUL from nonorthogonality
dDSX = zeros(HDIM,1);
dDSY = zeros(HDIM,1);
dDSZ = zeros(HDIM,1);
for Ia = 1:Nats % Derivatives Ra
  Ia_A = H_INDEX_START(Ia);
  Ia_B = H_INDEX_END(Ia);
  for iq = 1:HDIM
    dDSX(iq) = D(iq,Ia_A:Ia_B)*dSx(Ia_A:Ia_B,iq);
    dDSY(iq) = D(iq,Ia_A:Ia_B)*dSy(Ia_A:Ia_B,iq);
    dDSZ(iq) = D(iq,Ia_A:Ia_B)*dSz(Ia_A:Ia_B,iq);
  end
  for iq = Ia_A:Ia_B
    dDSX(iq) = dDSX(iq) + D(iq,:)*dSx(iq,:)';
    dDSY(iq) = dDSY(iq) + D(iq,:)*dSy(iq,:)';
    dDSZ(iq) = dDSZ(iq) + D(iq,:)*dSz(iq,:)';
  end
  for j = 1:Nats % Get the Mulliken charges for all atoms
    j_a = H_INDEX_START(j);
    j_b = H_INDEX_END(j);
    dQLxdR = sum(dDSX(j_a:j_b)); % Derivative with respect to Ia of charge on atom j
    dQLydR = sum(dDSY(j_a:j_b));
    dQLzdR = sum(dDSZ(j_a:j_b));
    FScoul(1,Ia) = FScoul(1,Ia) - dQLxdR*(U(j)*q(j) + CoulPot(j));
    FScoul(2,Ia) = FScoul(2,Ia) - dQLydR*(U(j)*q(j) + CoulPot(j));
    FScoul(3,Ia) = FScoul(3,Ia) - dQLzdR*(U(j)*q(j) + CoulPot(j));
  end

end
FScoul = 1*FScoul;

%CoulPot = C*q;  % Factor of 2 or 1/2 or +/-
FSspinA = zeros(3,Nats); % Coulomb force FSCOUL from nonorthogonality
dDSX = zeros(HDIM,1);
dDSY = zeros(HDIM,1);
dDSZ = zeros(HDIM,1);
for Ia = 1:Nats % Derivatives Ra
  Ia_A = H_INDEX_START(Ia);
  Ia_B = H_INDEX_END(Ia);
  for iq = 1:HDIM
    dDSX(iq) = Da(iq,Ia_A:Ia_B)*dSx(Ia_A:Ia_B,iq);
    dDSY(iq) = Da(iq,Ia_A:Ia_B)*dSy(Ia_A:Ia_B,iq);
    dDSZ(iq) = Da(iq,Ia_A:Ia_B)*dSz(Ia_A:Ia_B,iq);
  end
  for iq = Ia_A:Ia_B
    dDSX(iq) = dDSX(iq) + Da(iq,:)*dSx(iq,:)';
    dDSY(iq) = dDSY(iq) + Da(iq,:)*dSy(iq,:)';
    dDSZ(iq) = dDSZ(iq) + Da(iq,:)*dSz(iq,:)';
  end
  for j = 1:Nats % Get the Mulliken charges for all atoms
    j_a = H_INDEX_START(j);
    j_b = H_INDEX_END(j);
    dQLxdR = sum(dDSX(j_a:j_b)); % Derivative with respect to Ia of charge on atom j
    dQLydR = sum(dDSY(j_a:j_b));
    dQLzdR = sum(dDSZ(j_a:j_b));
    FSspinA(1,Ia) = FSspinA(1,Ia) - dQLxdR*(W(i)*NETSPIN(i));
    FSspinA(2,Ia) = FSspinA(2,Ia) - dQLydR*(W(i)*NETSPIN(i));
    FSspinA(3,Ia) = FSspinA(3,Ia) - dQLzdR*(W(i)*NETSPIN(i));
  end

end

%CoulPot = C*q;  % Factor of 2 or 1/2 or +/-
FSspinB = zeros(3,Nats); % Coulomb force FSCOUL from nonorthogonality
dDSX = zeros(HDIM,1);
dDSY = zeros(HDIM,1);
dDSZ = zeros(HDIM,1);
for Ia = 1:Nats % Derivatives Ra
  Ia_A = H_INDEX_START(Ia);
  Ia_B = H_INDEX_END(Ia);
  for iq = 1:HDIM
    dDSX(iq) = Db(iq,Ia_A:Ia_B)*dSx(Ia_A:Ia_B,iq);
    dDSY(iq) = Db(iq,Ia_A:Ia_B)*dSy(Ia_A:Ia_B,iq);
    dDSZ(iq) = Db(iq,Ia_A:Ia_B)*dSz(Ia_A:Ia_B,iq);
  end
  for iq = Ia_A:Ia_B
    dDSX(iq) = dDSX(iq) + Db(iq,:)*dSx(iq,:)';
    dDSY(iq) = dDSY(iq) + Db(iq,:)*dSy(iq,:)';
    dDSZ(iq) = dDSZ(iq) + Db(iq,:)*dSz(iq,:)';
  end
  for j = 1:Nats % Get the Mulliken charges for all atoms
    j_a = H_INDEX_START(j);
    j_b = H_INDEX_END(j);
    dQLxdR = sum(dDSX(j_a:j_b)); % Derivative with respect to Ia of charge on atom j
    dQLydR = sum(dDSY(j_a:j_b));
    dQLzdR = sum(dDSZ(j_a:j_b));
    FSspinB(1,Ia) = FSspinB(1,Ia) - dQLxdR*(W(i)*NETSPIN(i));
    FSspinB(2,Ia) = FSspinB(2,Ia) - dQLydR*(W(i)*NETSPIN(i));
    FSspinB(3,Ia) = FSspinB(3,Ia) - dQLzdR*(W(i)*NETSPIN(i));
  end

end
'''

In [ ]:
# Eb: -60.9249
# Ec: 0.0674
# s: -1.3544


In [ ]:
#ADF
# Eb: -60.80105204796506
# Ec: 0.06711797264107501
# s: -1.3131534719593752


0.06711797264107501

In [ ]:
0.00246654*27.21138625

0.06711797264107501

In [ ]:
const.shell_dim

Parameter containing:
tensor([0, 1, 3, 5], device='cuda:0')

In [ ]:
structure1.shell_types

tensor([1, 2, 1, 1, 2, 1, 1, 2, 3, 1, 2, 3], device='cuda:0')

In [ ]:
n_orb_per_shell[structure1.shell_types]

tensor([1, 3, 1, 1, 3, 1, 1, 3, 5, 1, 3, 5], device='cuda:0')

In [ ]:
n_orb_per_shell[structure1.shell_types]

tensor([1, 3, 1, 1, 3, 1, 1, 3, 5, 1, 3, 5], device='cuda:0')

In [ ]:
structure1.shell_types

tensor([1, 2, 1, 1, 2, 1, 1, 2, 3, 1, 2, 3], device='cuda:0')

In [ ]:
nu_shell = structure1.shell_types.unsqueeze(0).expand(len(structure1.shell_types), -1)
nu_atom = structure1.TYPE.repeat_interleave(structure1.n_shells_per_atom).unsqueeze(0).expand(len(structure1.shell_types), -1)

In [ ]:
mu_shell = structure1.shell_types.unsqueeze(0).expand(len(structure1.shell_types), -1).transpose(-1,-2)
mu_atom = structure1.TYPE.repeat_interleave(structure1.n_shells_per_atom).unsqueeze(0).expand(len(structure1.shell_types), -1).transpose(-1,-2)

In [ ]:
mu_atom.flatten()

tensor([6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [ ]:
def get_spin(S, net_spin, H_INDEX_START, n_orbitals_per_atom, ):
    for i = 1:Nr_atoms
        for j = 1:Nr_atoms
            for u = H_INDEX_START(i):H_INDEX_END(i)
                for v = H_INDEX_START(j):H_INDEX_END(j)
                    spinpol(u,v) = 0.5*S(u,v)*(net_spin(i)*W(i) + net_spin(j)*W(j));


In [ ]:
def get_spin(S, net_spin, H_INDEX_START, n_orbitals_per_atom, ):
    for i = 1:Nr_atoms
        for j = 1:Nr_atoms
            for u = H_INDEX_START(i):H_INDEX_END(i)
                for v = H_INDEX_START(j):H_INDEX_END(j)
                    spinpol(u,v) = 0.5*S(u,v)*(net_spin(i)*W(i) + net_spin(j)*W(j));
